## Importing Packages

In [67]:
import os
import sys
import pandas as pd
import nltk
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
import matplotlib.pyplot as plt
import string
import re
import numpy as np
import itertools
from nltk import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import ConfusionMatrixDisplay, recall_score,\
    accuracy_score, precision_score, f1_score, classification_report
import warnings
import xgboost

warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'xgboost'

In [2]:
df = pd.read_csv('fake reviews dataset.csv')
df.head()

,category,rating,label,text_
0,Home_and_Kitchen_5,5.0,CG,"Love this! Well made, sturdy, and very comfor..."
1,Home_and_Kitchen_5,5.0,CG,"love it, a great upgrade from the original. I..."
2,Home_and_Kitchen_5,5.0,CG,This pillow saved my back. I love the look and...
3,Home_and_Kitchen_5,1.0,CG,"Missing information on how to use it, but it i..."
4,Home_and_Kitchen_5,5.0,CG,Very nice set. Good quality. We have had the s...


## Data Undersatding

In [3]:
df['category'].value_counts()

Kindle_Store_5                  4730
Books_5                         4370
Pet_Supplies_5                  4254
Home_and_Kitchen_5              4056
Electronics_5                   3988
Sports_and_Outdoors_5           3946
Tools_and_Home_Improvement_5    3858
Clothing_Shoes_and_Jewelry_5    3848
Toys_and_Games_5                3794
Movies_and_TV_5                 3588
Name: category, dtype: int64

In [4]:
df.describe()

,rating
count,40432.000000
mean,4.256579
std,1.144354
min,1.000000
25%,4.000000
50%,5.000000
75%,5.000000
max,5.000000


In [5]:
df['target'] = np.where(df['label'] == 'CG', 1, 0)

In [6]:
df

,category,rating,label,text_,target
0,Home_and_Kitchen_5,5.0,CG,"Love this! Well made, sturdy, and very comfor...",1
1,Home_and_Kitchen_5,5.0,CG,"love it, a great upgrade from the original. I...",1
2,Home_and_Kitchen_5,5.0,CG,This pillow saved my back. I love the look and...,1
3,Home_and_Kitchen_5,1.0,CG,"Missing information on how to use it, but it i...",1
4,Home_and_Kitchen_5,5.0,CG,Very nice set. Good quality. We have had the s...,1
...,...,...,...,...,...
40427,Clothing_Shoes_and_Jewelry_5,4.0,OR,I had read some reviews saying that this bra r...,0
40428,Clothing_Shoes_and_Jewelry_5,5.0,CG,I wasn't sure exactly what it would be. It is ...,1
40429,Clothing_Shoes_and_Jewelry_5,2.0,OR,"You can wear the hood by itself, wear it with ...",0
40430,Clothing_Shoes_and_Jewelry_5,1.0,CG,I liked nothing about this dress. The only rea...,1


## Text Preprocessing: Tokenization

In [7]:
#tokenizing the text data in the 'text_' column of df
def tokenizer(x):
    
    corpus = [word_tokenize(doc) for doc in x]

# getting common stop words in english that we'll remove during tokenization/text normalization
    stop_words = stopwords.words('english')
    corpus_no_stopwords = []
    for words in corpus:
        docs = [x.lower() for x in words if ((x.isalpha()) & (x not in stop_words))]
        corpus_no_stopwords.append(docs)
    return corpus_no_stopwords

## Lemmantizer

In [8]:
def lemmatizer(corpus, as_string=True):
    lem = WordNetLemmatizer()
    
    def pos_tagger(nltk_tag):
        if nltk_tag.startswith('J'):
            return wordnet.ADJ
        elif nltk_tag.startswith('V'):
            return wordnet.VERB
        elif nltk_tag.startswith('N'):
            return wordnet.NOUN
        elif nltk_tag.startswith('R'):
            return wordnet.ADV
        else:         
            return None
    lemmatized_corpus = []
    for sentence in corpus:
        pos_tags = pos_tag(sentence)
        lemmatized_sentence = []
        for word, tag in pos_tags:
            pos = pos_tagger(tag)
            if pos is not None:
                lemmatized_word = lem.lemmatize(word, pos)
            else:
                lemmatized_word = lem.lemmatize(word)
            lemmatized_sentence.append(lemmatized_word)
        lemmatized_corpus.append(lemmatized_sentence)
    if as_string:
        lemmatized_corpus  = [' '.join(x) for x in lemmatized_corpus]
    return lemmatized_corpus
        

In [9]:
corpus_tokenized = tokenizer(df['text_'])

In [10]:
lemmatized_corpus = lemmatizer(corpus_tokenized)

## Pre vectorizing

In [11]:
joined_lemm_corpus = [' '.join(x) for x in lemmatized_corpus]
df['text_preproccesed'] = pd.Series(data=lemmatized_corpus)

In [12]:
vec = CountVectorizer(min_df = 0.05, max_df = 0.95)
X = vec.fit_transform(lemmatized_corpus)
countvec_df = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())

In [13]:
tfidf = TfidfVectorizer(min_df = 0.05, max_df = 0.95)
Y = tfidf.fit_transform(lemmatized_corpus)
tfidf_df = pd.DataFrame(Y.toarray(), columns=tfidf.get_feature_names())

In [14]:
tfidf_df

,also,anyone,best,big,bit,book,buy,ca,character,come,...,try,two,use,want,way,well,work,would,write,year
0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.365431,0.000000,0.000000,0.0,0.000000
1,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.728227
2,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.436464,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.000000,...,0.000000,0.350925,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40427,0.000000,0.0,0.112657,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.103073,...,0.115232,0.206830,0.000000,0.099786,0.000000,0.075666,0.000000,0.077660,0.0,0.000000
40428,0.000000,0.0,0.000000,0.091199,0.000000,0.0,0.13296,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.061434,0.067497,0.252213,0.0,0.000000
40429,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.000000,...,0.174608,0.000000,0.000000,0.000000,0.000000,0.114655,0.000000,0.117676,0.0,0.000000
40430,0.064548,0.0,0.000000,0.000000,0.145785,0.0,0.11615,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.107335,0.000000,0.055082,0.0,0.000000


In [15]:
X

<40432x89 sparse matrix of type '<class 'numpy.int64'>'
	with 375538 stored elements in Compressed Sparse Row format>

In [16]:
X_train, X_test, y_train,y_test = train_test_split(df['text_'],df['target'], test_size=0.3,random_state=42)

In [17]:
X_train_preprocessed = lemmatizer(tokenizer(X_train))
X_test_preprocessed = lemmatizer(tokenizer(X_test))

### Try Random Forest and Logistic Regression with the Count Vectorizer

In [18]:
def scores(y_test, y_pred):
    print(f"""    Accuracy: {round(accuracy_score(y_test,y_pred),3)}
    
    Recall: {round(recall_score(y_test,y_pred),3)}
    
    F1: {round(f1_score(y_test,y_pred),3)}
    
    Precision: {round(precision_score(y_test,y_pred),3)}""")

In [ ]:
steps = [('countvec',CountVectorizer(min_df = 0.05, max_df = 0.95)),('rfc',RandomForestClassifier(n_estimators=200,random_state=42))]
pipe_cv_rf = Pipeline(steps)
pipe_cv_rf.fit(X_train_preprocessed, y_train)
pipe_cv_rf.score(X_train_preprocessed,y_train)

In [54]:
crossval_rf_cv = np.mean(cross_val_score(pipe_cv_rf, X_train_preprocessed,y_train,scoring='accuracy',cv=5))

In [59]:
crossval_rf_cv

0.7636212278746288

In [ ]:
steps[1] = ('logreg',LogisticRegression(random_state=42))
pipe_cv_lr = Pipeline(steps)
pipe_cv_lr.fit(X_train_preprocessed, y_train)
pipe_cv_lr.score(X_train_preprocessed,y_train)

In [55]:
crossval_lr_cv = np.mean(cross_val_score(pipe_cv_lr, X_train_preprocessed,y_train,scoring='accuracy',cv=5))

In [60]:
crossval_lr_cv

0.7266269054338063

In [ ]:
steps = [('tfidfvec',TfidfVectorizer(min_df = 0.05, max_df = 0.95)),('rfc',RandomForestClassifier(n_estimators=200,random_state=42))]
pipe_idf_rf = Pipeline(steps)
pipe_idf_rf.fit(X_train_preprocessed, y_train)
pipe_idf_rf.score(X_train_preprocessed,y_train)

In [56]:
crossval_rf_idf = np.mean(cross_val_score(pipe_idf_rf, X_train_preprocessed,y_train,scoring='accuracy',cv=5))

In [61]:
crossval_rf_idf

0.7631969029932032

In [ ]:
steps[1] = ('logreg',LogisticRegression(random_state=42))
pipe_idf_lr = Pipeline(steps)
pipe_idf_lr.fit(X_train_preprocessed, y_train)
pipe_idf_lr.score(X_train_preprocessed,y_train)

In [57]:
crossval_lr_idf = np.mean(cross_val_score(pipe_idf_lr, X_train_preprocessed,y_train,scoring='accuracy',cv=5))

In [62]:
crossval_lr_idf

0.7267683043675561

We run two models with both types of vectorizers, and based on our scores, we decided to stick with the Count Vectorizer.

In [31]:
steps_rf = [('countvec',CountVectorizer(min_df = 0.05, max_df = 0.95)),('rfc',RandomForestClassifier(n_estimators=200,random_state=42))]
steps_lr = [('countvec',CountVectorizer(min_df = 0.05, max_df = 0.95)),('logreg',LogisticRegression(random_state=42,max_iter=10000))]

pipe_rf = Pipeline(steps_rf)
pipe_lr = Pipeline(steps_lr)

In [32]:
param_rf = {'rfc__n_estimators':[100,150,200],
                       'rfc__max_depth':[2,3,4,5],
                       'rfc__min_samples_leaf':[1,2,3]
                      }

param_lr = {'logreg__penalty':['l1', 'l2', 'elasticnet', None],
                 'logreg__C':[0.001,0.01,0.1,1,10,100],
                 'logreg__solver':['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']
                }

In [33]:
gs_rf = GridSearchCV(estimator=pipe_rf, param_grid=param_rf,scoring='accuracy',cv=5,error_score=0)
gs_lr = GridSearchCV(estimator=pipe_lr, param_grid=param_lr,scoring='accuracy',cv=5,error_score=0)
gs_rf.fit(X_train_preprocessed,y_train)
gs_lr.fit(X_train_preprocessed,y_train)

C:\Users\eduar\anaconda3\envs\new_env\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to 0.000000. Details: 
Traceback (most recent call last):
  File "C:\Users\eduar\anaconda3\envs\new_env\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\eduar\anaconda3\envs\new_env\lib\site-packages\sklearn\pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\eduar\anaconda3\envs\new_env\lib\site-packages\sklearn\linear_model\_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\eduar\anaconda3\envs\new_env\lib\site-packages\sklearn\linear_model\_logistic.py", line 443, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "

GridSearchCV(cv=5, error_score=0,
             estimator=Pipeline(steps=[('countvec',
                                        CountVectorizer(max_df=0.95,
                                                        min_df=0.05)),
                                       ('logreg',
                                        LogisticRegression(max_iter=10000,
                                                           random_state=42))]),
             param_grid={'logreg__C': [0.001, 0.01, 0.1, 1, 10, 100],
                         'logreg__penalty': ['l1', 'l2', 'elasticnet', None],
                         'logreg__solver': ['lbfgs', 'liblinear', 'newton-cg',
                                            'newton-cholesky', 'sag', 'saga']},
             scoring='accuracy')

In [34]:
gs_rf.best_params_

{'rfc__max_depth': 5, 'rfc__min_samples_leaf': 2, 'rfc__n_estimators': 100}

In [35]:
gs_lr.best_params_

{'logreg__C': 1, 'logreg__penalty': 'l1', 'logreg__solver': 'liblinear'}

In [36]:
gs_rf.best_score_

0.7035902146170281

In [38]:
gs_lr.best_score_

0.7268742427732242

In [63]:
steps = [('countvec',CountVectorizer(min_df = 0.05, max_df = 0.95)),('ada',AdaBoostClassifier())]
pipe_ada_booster = Pipeline(steps).fit(X_train_preprocessed,y_train)
pipe_ada_booster.score(X_train_preprocessed,y_train)

0.7291357501236662

In [64]:
crossval_ada_booster = np.mean(cross_val_score(pipe_ada_booster, X_train_preprocessed,y_train,scoring='accuracy',cv=5))
crossval_ada_booster

0.7228819653159707

In [65]:
gbc = ('gbc',GradientBoostingClassifier())
steps[1] = gbc
pipe_gbc = Pipeline(steps).fit(X_train_preprocessed,y_train)
pipe_gbc.score(X_train_preprocessed,y_train)

0.7461310154759381

In [66]:
crossval_gradient_booster = np.mean(cross_val_score(pipe_gbc, X_train_preprocessed,y_train,scoring='accuracy',cv=5))
crossval_gradient_booster

0.7360966953234673